## Download Financial data from List

Objectives:
- Open list from json file
- Create an error file
- Use the list to download financial data
- Log in error list any tickers with problems

In [1]:
# Imports
import os
import requests
import pandas as pd
import json
from sqlalchemy import create_engine

In [2]:
# Importing API Key
api = os.environ.get("token_finmodelprep2")

In [ ]:
# Variables
fin_data = '/income-statement/'
sqlite_table = "income_statement"
# fin_data = '/balance-sheet-statement/'
# sqlite_table = "balance_sheet"
# fin_data = '/cash-flow-statement/'
# sqlite_table = "cashflow_statement"

In [ ]:
# Open Json File and save as variable
with open("ticker_list.json", 'r') as f:
    ticker_list = json.load(f)

In [ ]:
# Create an Error file
f = open("error_list.txt", "w")
f.write("ErrorList:\n")
f.close()

In [ ]:
# Importing data from Financial Modeling Prep
for company in ticker_list:
    print(company)
    try:
        # Request financial data
        www = 'https://financialmodelingprep.com/api/v3'
        fmp_data = requests.get(f'{www}{fin_data}{company}?apikey={api}').json()
        
        # Save financial data as Dataframe
        df = pd.DataFrame(fmp_data)

        # Transform column from string to date format
        df['date'] = pd.to_datetime(df['date'])
        df['fillingDate'] = pd.to_datetime(df['fillingDate'])
        df['acceptedDate'] = pd.to_datetime(df['acceptedDate'])
    
        # Create an engine
        engine = create_engine('sqlite:///financialmodelingprep.db', echo=True)
        sqlite_connection = engine.connect()

        # Create a table
        df.to_sql(sqlite_table, sqlite_connection, if_exists='append', index=False)

        # Close connection
        sqlite_connection.close()
    except:
        f = open("error_list.txt", "a")
        f.write("\n"+company)
        f.close()
        pass

### Manual Version

In [9]:
# Variables
company = "ALMIL.PA"

# fin_data = '/income-statement/'
# fin_data = '/balance-sheet-statement/'
fin_data = '/cash-flow-statement/'

sqlite_table = "income_statement"
# sqlite_table = "balance_sheet"
# sqlite_table = "cashflow_statement"

In [10]:
# Request Financial Data
www = 'https://financialmodelingprep.com/api/v3'
fmp_data = requests.get(f'{www}{fin_data}{company}?apikey={api}').json()

In [11]:
# Save Financial Data as Dataframe
df = pd.DataFrame(fmp_data)

# Transform column from string to date format
df['date'] = pd.to_datetime(df['date'])
df['fillingDate'] = pd.to_datetime(df['fillingDate'])
df['acceptedDate'] = pd.to_datetime(df['acceptedDate'])

df.dtypes
df.head()

,date,symbol,reportedCurrency,cik,fillingDate,acceptedDate,calendarYear,period,netIncome,depreciationAndAmortization,...,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow,link,finalLink
0,2020-12-31,ALMIL.PA,EUR,None,2020-12-31,2020-12-31,2020,FY,-6796000.0,4912000.0,...,-2069000.0,-75000.0,6156000.0,19882000,13726000,10435000.0,-2246000.0,8189000.0,https://www.infogreffe.com/recherche-entrepris...,https://www.infogreffe.com/recherche-entrepris...
1,2019-12-31,ALMIL.PA,EUR,None,2019-12-31,2019-12-31,2019,FY,2502000.0,4535000.0,...,-4136000.0,0.0,-4896000.0,13726000,18622000,2764000.0,-3550000.0,-786000.0,https://www.infogreffe.com/recherche-entrepris...,https://www.infogreffe.com/recherche-entrepris...
2,2018-12-31,ALMIL.PA,EUR,None,2018-12-31,2018-12-31,2018,FY,4239000.0,3517000.0,...,-12763000.0,0.0,-13685000.0,18622000,32307000,2607000.0,-3476000.0,-869000.0,https://www.infogreffe.com/recherche-entrepris...,https://www.infogreffe.com/recherche-entrepris...
3,2017-12-31,ALMIL.PA,EUR,None,2017-12-31,2017-12-31,2017,FY,5280000.0,2906000.0,...,-8077000.0,-164000.0,-4734000.0,32307000,37041000,6523000.0,-2927000.0,3596000.0,https://www.infogreffe.com/recherche-entrepris...,https://www.infogreffe.com/recherche-entrepris...
4,2016-12-31,ALMIL.PA,EUR,None,2016-12-31,2016-12-31,2016,FY,6108000.0,2208000.0,...,-6931000.0,-75000.0,3035000.0,37041000,34006000,12302000.0,-2480000.0,9822000.0,https://www.infogreffe.com/recherche-entrepris...,https://www.infogreffe.com/recherche-entrepris...


In [ ]:
# Create an engine
engine = create_engine('sqlite:///financialmodelingprep.db', echo=False)
sqlite_connection = engine.connect()
# sqlite is the database type
# financialmodelingprep.db is the sqlite db name
# echo=True to see output from database connection

# Create a table
df.to_sql(sqlite_table, sqlite_connection, if_exists='append', index=False)
# Alternativas:
    # if_exists: ‘fail’, ‘replace’, ‘append’
        # fail: Raise a ValueError
        # replace: Drop the table before inserting new values.
        # append: Insert new values to the existing table
    # index: bool, default True.
        # Write DataFrame index as a column. 
        # Uses index_label as the column name in the table.

# Close connection
sqlite_connection.close()